In [223]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import geopandas as gpd
#import geopy?
%matplotlib inline

In [268]:
def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

In [224]:
acs_datapath = "data_v2/Dept_37-00049/37-00049_ACS_data/"

In [225]:
education_data = "37-00049_ACS_education-attainment/ACS_16_5YR_S1501_with_ann.csv"

In [226]:
education_df = pd.read_csv(acs_datapath + education_data, header = 0, skiprows=[1])
sub_header = pd.read_csv(acs_datapath + education_data, header = 1).keys()

In [254]:
print(education_df.shape)
education_df["GEO.id2"] = pd.to_numeric(education_df["GEO.id2"])

c = list(education_df.columns)
c[3:] = ["education_" + c_name for c_name in c[3:]]
education_df.columns = c

education_df.head()

(529, 771)


,GEO.id,GEO.id2,GEO.display-label,education_education_HC01_EST_VC02,education_education_HC01_MOE_VC02,education_education_HC02_EST_VC02,education_education_HC02_MOE_VC02,education_education_HC03_EST_VC02,education_education_HC03_MOE_VC02,education_education_HC04_EST_VC02,...,education_education_HC02_EST_VC85,education_education_HC02_MOE_VC85,education_education_HC03_EST_VC85,education_education_HC03_MOE_VC85,education_education_HC04_EST_VC85,education_education_HC04_MOE_VC85,education_education_HC05_EST_VC85,education_education_HC05_MOE_VC85,education_education_HC06_EST_VC85,education_education_HC06_MOE_VC85
0,1400000US48113000100,48113000100,"Census Tract 1, Dallas County, Texas",195,101,(X),(X),90,69,(X),...,(X),(X),178750,97008,(X),(X),91507,10438,(X),(X)
1,1400000US48113000201,48113000201,"Census Tract 2.01, Dallas County, Texas",36,25,(X),(X),36,25,(X),...,(X),(X),128182,33333,(X),(X),76875,16293,(X),(X)
2,1400000US48113000202,48113000202,"Census Tract 2.02, Dallas County, Texas",217,95,(X),(X),82,86,(X),...,(X),(X),101824,14028,(X),(X),57813,13941,(X),(X)
3,1400000US48113000300,48113000300,"Census Tract 3, Dallas County, Texas",262,176,(X),(X),176,138,(X),...,(X),(X),71250,36939,(X),(X),54375,21490,(X),(X)
4,1400000US48113000401,48113000401,"Census Tract 4.01, Dallas County, Texas",740,194,(X),(X),283,113,(X),...,(X),(X),58125,21588,(X),(X),55735,4681,(X),(X)


In [255]:
FIPS_state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

In [256]:
tx_census_shape_dir = "census_shape/tl_2016_48_tract/"
tx_census_shp = "tl_2016_48_tract.shp"

In [257]:
tx_census_shp_df = gpd.read_file(tx_census_shape_dir + tx_census_shp)
tx_census_shp_df["GEOID"] = pd.to_numeric(tx_census_shp_df["GEOID"])
tx_census_shp_df.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,48,201,542301,48201542301,5423.01,Census Tract 5423.01,G5020,S,5956745,15456,+29.8204990,-095.7316493,"POLYGON ((-95.75223799999999 29.831686, -95.75..."
1,48,201,552601,48201552601,5526.01,Census Tract 5526.01,G5020,S,2911911,4263,+29.9629845,-095.5679180,"POLYGON ((-95.585241 29.966226, -95.5852209999..."
2,48,157,673800,48157673800,6738,Census Tract 6738,G5020,S,14100089,1008190,+29.5874575,-095.7125243,"POLYGON ((-95.758386 29.583531, -95.757993 29...."
3,48,157,674000,48157674000,6740,Census Tract 6740,G5020,S,4826110,65291,+29.5869509,-095.6224534,"POLYGON ((-95.63467299999999 29.589525, -95.63..."
4,48,157,674100,48157674100,6741,Census Tract 6741,G5020,S,4170610,0,+29.5791692,-095.5995891,"POLYGON ((-95.614667 29.578282, -95.6133859999..."


In [258]:
tx_census_shp_df.keys()
#print(type(tx_census_shp_df["geometry"]))

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [259]:
education_with_geo = education_df.merge(tx_census_shp_df, left_on='GEO.id2', right_on='GEOID', how='left')

geometry = education_with_geo["geometry"]
education_with_geo.drop(["geometry"], axis = 1)
education_with_geo = gpd.GeoDataFrame(education_with_geo, crs=tx_census_shp_df["geometry"].crs, geometry=geometry)

In [260]:
education_with_geo.head()

,GEO.id,GEO.id2,GEO.display-label,education_education_HC01_EST_VC02,education_education_HC01_MOE_VC02,education_education_HC02_EST_VC02,education_education_HC02_MOE_VC02,education_education_HC03_EST_VC02,education_education_HC03_MOE_VC02,education_education_HC04_EST_VC02,...,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,1400000US48113000100,48113000100,"Census Tract 1, Dallas County, Texas",195,101,(X),(X),90,69,(X),...,48113000100,1,Census Tract 1,G5020,S,3064914,511269,+32.8166094,-096.7365761,"POLYGON ((-96.75300799999999 32.815674, -96.75..."
1,1400000US48113000201,48113000201,"Census Tract 2.01, Dallas County, Texas",36,25,(X),(X),36,25,(X),...,48113000201,2.01,Census Tract 2.01,G5020,S,1762502,0,+32.8327403,-096.7571634,"POLYGON ((-96.76156399999999 32.829894, -96.76..."
2,1400000US48113000202,48113000202,"Census Tract 2.02, Dallas County, Texas",217,95,(X),(X),82,86,(X),...,48113000202,2.02,Census Tract 2.02,G5020,S,1571513,30513,+32.8318688,-096.7662373,"POLYGON ((-96.770189 32.82906, -96.770167 32.8..."
3,1400000US48113000300,48113000300,"Census Tract 3, Dallas County, Texas",262,176,(X),(X),176,138,(X),...,48113000300,3,Census Tract 3,G5020,S,1809161,0,+32.8301654,-096.7761239,"POLYGON ((-96.78599 32.822262, -96.78561099999..."
4,1400000US48113000401,48113000401,"Census Tract 4.01, Dallas County, Texas",740,194,(X),(X),283,113,(X),...,48113000401,4.01,Census Tract 4.01,G5020,S,2032889,0,+32.8113683,-096.8294337,"POLYGON ((-96.84267 32.81611, -96.842361999999..."


In [261]:
dallas_police_shape_path = "data_v2/Dept_37-00049/37-00049_Shapefiles/EPIC.shp"

In [262]:
dallas_police_shape_df = gpd.read_file(dallas_police_shape_path)
dallas_police_shape_df.head()

dallas_police_shape_df["geometry"] = dallas_police_shape_df["geometry"].to_crs(education_with_geo["geometry"].crs)
dallas_police_shape_df.crs = education_with_geo.crs

dallas_police_shape_df.head()

,Name,Shape_STAr,Shape_STLe,geometry
0,EPIC_Fair Park,9.312213e+07,70821.503172,"POLYGON ((-96.7373391444538 32.79105860853344,..."
1,EPIC_CampWisdomCorridor,1.139695e+08,57536.951272,"POLYGON ((-96.8632907544893 32.67178827185337,..."
2,EPIC West Dallas,1.258257e+08,45373.481738,POLYGON ((-96.84478909451936 32.76384812895372...
3,EPIC Vickery Meadow,5.226778e+07,30305.941933,"POLYGON ((-96.75677506831076 32.8835155665009,..."
4,EPIC_Lancaster Corridor,7.831274e+07,39182.926747,"POLYGON ((-96.77515500053748 32.7129630002214,..."


In [263]:
census_police_district_id = []
census_police_district_percent_overlap = []

for i in range(education_with_geo.shape[0]):
    g = education_with_geo["geometry"][i]
    list_of_matches = []
    perc_overlap = []
    for j in range(dallas_police_shape_df.shape[0]):
        a = dallas_police_shape_df["geometry"][j].intersection(g)
        if a.area != 0:
            percent_overlap = 100 * (a.area / g.area)
            list_of_matches.append(dallas_police_shape_df["Name"][j])
            perc_overlap.append(percent_overlap)
    census_police_district_id.append(tuple(list_of_matches))
    census_police_district_percent_overlap.append(tuple(perc_overlap))   
    
education_with_geo["Police District ID"] = pd.Series(census_police_district_id)
education_with_geo["Percentage Overlap"] = pd.Series(census_police_district_percent_overlap)

In [264]:
education_with_geo = education_with_geo[education_with_geo["Police District ID"] != ()]

In [265]:
print(education_with_geo.shape)
education_with_geo.head()

(53, 786)


,GEO.id,GEO.id2,GEO.display-label,education_education_HC01_EST_VC02,education_education_HC01_MOE_VC02,education_education_HC02_EST_VC02,education_education_HC02_MOE_VC02,education_education_HC03_EST_VC02,education_education_HC03_MOE_VC02,education_education_HC04_EST_VC02,...,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,Police District ID,Percentage Overlap
36,1400000US48113002000,48113002000,"Census Tract 20, Dallas County, Texas",677,250,(X),(X),304,221,(X),...,Census Tract 20,G5020,S,2393483,62043,+32.7597444,-096.8173369,"POLYGON ((-96.82293399999999 32.756332, -96.82...","(EPIC West Dallas,)","(0.01154741048693726,)"
39,1400000US48113002400,48113002400,"Census Tract 24, Dallas County, Texas",287,99,(X),(X),106,72,(X),...,Census Tract 24,G5020,S,1255975,0,+32.7939108,-096.7573364,"POLYGON ((-96.76912299999999 32.785823, -96.76...","(EPIC_Fair Park,)","(0.22407386683504088,)"
40,1400000US48113002500,48113002500,"Census Tract 25, Dallas County, Texas",426,148,(X),(X),198,94,(X),...,Census Tract 25,G5020,S,4202733,0,+32.7853521,-096.7452474,"POLYGON ((-96.76833499999999 32.784686, -96.76...","(EPIC_Fair Park,)","(69.68153933497673,)"
41,1400000US48113002701,48113002701,"Census Tract 27.01, Dallas County, Texas",370,168,(X),(X),245,137,(X),...,Census Tract 27.01,G5020,S,1867998,386013,+32.7738368,-096.7396835,"POLYGON ((-96.751527 32.776972, -96.751176 32....","(EPIC_Fair Park,)","(51.32021146534651,)"
42,1400000US48113002702,48113002702,"Census Tract 27.02, Dallas County, Texas",184,64,(X),(X),103,55,(X),...,Census Tract 27.02,G5020,S,1069619,0,+32.7701729,-096.7507889,"POLYGON ((-96.75988099999999 32.770313, -96.75...","(EPIC_Fair Park,)","(87.12400720332192,)"


In [266]:
centering_coord = [float(education_with_geo["INTPTLAT"].values[0]), float(education_with_geo["INTPTLON"].values[0])]

In [270]:
mapa = folium.Map(centering_coord, height=400, zoom_start=10, tiles='Stamen Toner')
folium.GeoJson(education_with_geo, style_function= lambda x:{'color':'blue'}).add_to(mapa)
folium.GeoJson(dallas_police_shape_df, style_function= lambda x:{'color':'red'}).add_to(mapa)
mapa
embed_map(mapa)

In [271]:
#poverty, education, race sex age

In [272]:
race_sex_age_path = "37-00049_ACS_race-sex-age/ACS_15_5YR_DP05_with_ann.csv"
poverty_path = "37-00049_ACS_poverty/ACS_16_5YR_S1701_with_ann.csv"

In [273]:
race_sex_age_df = pd.read_csv(acs_datapath + race_sex_age_path, header = 0, skiprows=[1])
poverty_df = pd.read_csv(acs_datapath + poverty_path, header = 0, skiprows=[1])

race_sex_age_df["GEO.id2"] = pd.to_numeric(race_sex_age_df["GEO.id2"])
poverty_df["GEO.id2"] = pd.to_numeric(poverty_df["GEO.id2"])

c = list(race_sex_age_df.columns)
c[3:] = ["race_sex_age_" + c_name for c_name in c[3:]]
race_sex_age_df.columns = c

c = list(poverty_df.columns)
c[3:] = ["poverty_" + c_name for c_name in c[3:]]
poverty_df.columns = c

In [277]:
census_master_df = education_with_geo.merge(race_sex_age_df, left_on='GEO.id2', right_on='GEO.id2', how='left')
census_master_df = master_df.merge(poverty_df, left_on='GEO.id2', right_on='GEO.id2', how='left')

In [278]:
census_master_df.head()

,GEO.id_x,GEO.id2,GEO.display-label_x,education_education_HC01_EST_VC02,education_education_HC01_MOE_VC02,education_education_HC02_EST_VC02,education_education_HC02_MOE_VC02,education_education_HC03_EST_VC02,education_education_HC03_MOE_VC02,education_education_HC04_EST_VC02,...,poverty_HC02_EST_VC78_y,poverty_HC02_MOE_VC78_y,poverty_HC03_EST_VC78_y,poverty_HC03_MOE_VC78_y,poverty_HC01_EST_VC79_y,poverty_HC01_MOE_VC79_y,poverty_HC02_EST_VC79_y,poverty_HC02_MOE_VC79_y,poverty_HC03_EST_VC79_y,poverty_HC03_MOE_VC79_y
0,1400000US48113002000,48113002000,"Census Tract 20, Dallas County, Texas",677,250,(X),(X),304,221,(X),...,140,106,58.8,26.1,397,145,294,136,74.1,15.1
1,1400000US48113002400,48113002400,"Census Tract 24, Dallas County, Texas",287,99,(X),(X),106,72,(X),...,34,39,33.3,36.4,187,94,159,92,85.0,14.9
2,1400000US48113002500,48113002500,"Census Tract 25, Dallas County, Texas",426,148,(X),(X),198,94,(X),...,25,32,16.6,21.8,505,149,339,121,67.1,11.6
3,1400000US48113002701,48113002701,"Census Tract 27.01, Dallas County, Texas",370,168,(X),(X),245,137,(X),...,76,59,47.5,29.4,341,137,255,128,74.8,13.7
4,1400000US48113002702,48113002702,"Census Tract 27.02, Dallas County, Texas",184,64,(X),(X),103,55,(X),...,35,31,85.4,21.5,264,77,237,77,89.8,7.2


In [276]:
#how to stratify by police districts

In [283]:
where_id = census_master_df["Police District ID"].apply(lambda x: "EPIC West Dallas" in x)
census_master_df[where_id == True]

,GEO.id_x,GEO.id2,GEO.display-label_x,education_education_HC01_EST_VC02,education_education_HC01_MOE_VC02,education_education_HC02_EST_VC02,education_education_HC02_MOE_VC02,education_education_HC03_EST_VC02,education_education_HC03_MOE_VC02,education_education_HC04_EST_VC02,...,poverty_HC02_EST_VC78_y,poverty_HC02_MOE_VC78_y,poverty_HC03_EST_VC78_y,poverty_HC03_MOE_VC78_y,poverty_HC01_EST_VC79_y,poverty_HC01_MOE_VC79_y,poverty_HC02_EST_VC79_y,poverty_HC02_MOE_VC79_y,poverty_HC03_EST_VC79_y,poverty_HC03_MOE_VC79_y
0,1400000US48113002000,48113002000,"Census Tract 20, Dallas County, Texas",677,250,(X),(X),304,221,(X),...,140,106,58.8,26.1,397,145,294,136,74.1,15.1
9,1400000US48113004201,48113004201,"Census Tract 42.01, Dallas County, Texas",336,150,(X),(X),244,126,(X),...,1,3,0.5,1.5,269,145,170,129,63.2,26.8
10,1400000US48113004300,48113004300,"Census Tract 43, Dallas County, Texas",177,75,(X),(X),132,71,(X),...,16,24,47.1,50.6,109,56,97,54,89.0,10.8
11,1400000US48113004400,48113004400,"Census Tract 44, Dallas County, Texas",188,115,(X),(X),82,67,(X),...,87,55,36.4,18.1,123,58,52,38,42.3,24.0
14,1400000US48113006900,48113006900,"Census Tract 69, Dallas County, Texas",452,152,(X),(X),265,115,(X),...,78,77,72.9,25.4,508,125,342,126,67.3,11.7
33,1400000US48113010101,48113010101,"Census Tract 101.01, Dallas County, Texas",573,167,(X),(X),300,143,(X),...,11,18,40.7,40.7,263,135,211,110,80.2,14.1
34,1400000US48113010102,48113010102,"Census Tract 101.02, Dallas County, Texas",250,87,(X),(X),76,45,(X),...,38,34,47.5,52.5,34,28,17,21,50.0,37.3
52,1400000US48113020500,48113020500,"Census Tract 205, Dallas County, Texas",453,104,(X),(X),166,81,(X),...,21,20,24.4,24.5,488,87,390,87,79.9,9.9
